Copyright (c) 2025–2026 Quan-feng WU <wuquanfeng@ihep.ac.cn>

This notebook is released under the [MIT License](https://opensource.org/licenses/MIT).

# Preliminaries

In [ ]:
using ArbNumerics
using CairoMakie
using DataInterpolations
using DelimitedFiles
using DifferentialEquations
using Format
using JLD2
using LaTeXStrings
using SpecialFunctions
using LambertW

using NaturalUnits

In [ ]:
natural_format = "%.2e"
float_format = "%.2f"

set_theme!(theme_latexfonts())

In [ ]:
arbfloat_1000(x) = ArbFloat(x; bits=1000)

In [ ]:
include("tool_script-ArbNumerics_compat.jl")
include("tool_script-directories.jl")
include("tool_script-geomspace.jl")

include("tool_script-N_energy_density.jl")
include("tool_script-branching_ratios.jl")
include("tool_script-sterile_neutrino_production.jl")

include("tool_script-integrals.jl")
include("tool_script-parameters.jl")

@info "Loaded all tool scripts."

# Evolution Systems

In [ ]:
function evolution_system_SBBN!(du, u, p, η)
    a = exp(η)
    log_Xₙ, log_ρ_SM_in_EU = u

    Xₙ = exp(log_Xₙ)
    ρ_SM = EU(exp(log_ρ_SM_in_EU), 4)

    T_SM = from_ρ_SM_to_T_SM(ρ_SM)

    x = Q / T_SM
    Γₙ_SM = (255 / τ_n) * (12 + 6 * x + x^2) / x^5

    ### Xₙ evolution ##################################
    d_Xₙ_d_t_SM = -Γₙ_SM * (Xₙ - (1 - Xₙ) * exp(-x))
    d_Xₙ_d_t_decay = -Xₙ / τ_n

    d_Xₙ_d_t = d_Xₙ_d_t_SM + d_Xₙ_d_t_decay
    ##################################################

    ### energy density background and Hubble parameter
    H = sqrt(ρ_SM / (3 * M_Pl^2))

    d_log_ρ_SM_d_η = -4
    ##################################################

    du[1] = d_Xₙ_d_t / (H * Xₙ)
    du[2] = d_log_ρ_SM_d_η
    return du
end

evolution_problem_SBBN = ODEProblem(evolution_system_SBBN!,
    map(log ∘ (v -> EUval(EU, v)),
        [inv(1 + exp(Q / GeV(10))), (π^2 / 30) * N_ρ(GeV(10)) * GeV(10)^4]
    ),
    (0, log(1e7))
)
evolution_solution_SBBN = solve(evolution_problem_SBBN, AutoTsit5(Rosenbrock23()))

a_list_SBBN = geomspace(1, 1e7, 1000)
results_list_SBBN = (evolution_solution_SBBN ∘ log).(a_list_SBBN)
ρ_SM_list_SBBN = map((x -> EU(x, 4)) ∘ exp ∘ last, results_list_SBBN)
T_SM_list_SBBN = from_ρ_SM_to_T_SM.(ρ_SM_list_SBBN)
Xₙ_list_SBBN = (exp ∘ first).(results_list_SBBN)

@info "SBBN scenario done."

In [ ]:
function evolution_system!(du, u, p, η)
    a = exp(η)
    log_t_in_EU, log_Xₙ,
        log_ρ_SM_in_EU, log_ρ₄_in_EU, log_ρₓ_in_EU,
        log_n_pion_minus_in_EU, log_n_pion_plus_in_EU,
        log_n_kaon_minus_in_EU, log_n_kaon_zero_long_in_EU = u
    m₄, U_a4_norm, Γ₄, Γₓ = p

    t = EU(exp(log_t_in_EU), -1)
    Xₙ = exp(log_Xₙ)
    ρ_SM = EU(exp(log_ρ_SM_in_EU), 4)
    ρ₄ = EU(exp(log_ρ₄_in_EU < -300 ? -300 : log_ρ₄_in_EU), 4)
    ρₓ = EU(exp(log_ρₓ_in_EU), 4)
    n_pion_minus = EU(exp(log_n_pion_minus_in_EU < -300 ? -300 : log_n_pion_minus_in_EU), 3)
    n_pion_plus = EU(exp(log_n_pion_plus_in_EU < -300 ? -300 : log_n_pion_plus_in_EU), 3)
    n_kaon_minus = EU(exp(log_n_kaon_minus_in_EU < -300 ? -300 : log_n_kaon_minus_in_EU), 3)
    n_kaon_zero_long = EU(exp(log_n_kaon_zero_long_in_EU < -300 ? -300 : log_n_kaon_zero_long_in_EU), 3)

    ρ_tot = ρ_SM + ρ₄ + ρₓ
    T_SM = from_ρ_SM_to_T_SM(ρ_SM)

    this_BKG_coeff = BKG_coeff(T_SM, m₄, U_a4_norm, c_ν_e)

    ρ₄_eq = g₄ * numeric_energy_density_FD(T_SM / m₄) * m₄^4
    T₄ = numeric_Tₘ_FD(ρ₄ / (g₄ * m₄^4)) * m₄
    P₄ = iszero(T₄) ? zero(ρ₄) : g₄ * numeric_pressure_FD(T₄ / m₄) * m₄^4
    n₄ = iszero(T₄) ? ρ₄ / m₄ : numeric_number_density_FD(T₄ / m₄) * m₄^3

    Tₓ = (sqrt ∘ sqrt)(ρₓ / (gₓ * π^2 / 30))

    w₄ = P₄ / ρ₄
    # @show w₄, T₄ / m₄

    γ₄ = ρ₄ / (m₄ * n₄)
    Γ₄ /= γ₄
    Γₓ /= γ₄

    n_γ = 2 * ζ₃ * T_SM^3 / π^2
    n_B = η_B * n_γ

    x = Q / T_SM
    Γₙ_SM = (255 / τ_n) * (12 + 6 * x + x^2) / x^5

    ### Xₙ evolution ##################################
    d_Xₙ_d_t_SM = -Γₙ_SM * (Xₙ - (1 - Xₙ) * exp(-x))
    d_Xₙ_d_t_decay = -Xₙ / τ_n
    d_Xₙ_d_t_pion = (1 - Xₙ) * n_pion_minus * σv_pion_minus_pton(T_SM) - Xₙ * n_pion_plus * σv_pion_plus_ntop(T_SM)
    d_Xₙ_d_t_kaon_minus = (1 - Xₙ) * n_kaon_minus * σv_kaon_minus_pton(T_SM) - Xₙ * n_kaon_minus * σv_kaon_minus_ntop(T_SM)
    d_Xₙ_d_t_kaon_zero_long = (1 - Xₙ) * n_kaon_zero_long * σv_kaon_zero_long_pton(T_SM) - Xₙ * n_kaon_zero_long * σv_kaon_zero_long_ntop(T_SM)

    d_Xₙ_d_t = d_Xₙ_d_t_SM + d_Xₙ_d_t_decay +
                d_Xₙ_d_t_pion + d_Xₙ_d_t_kaon_minus +
                d_Xₙ_d_t_kaon_zero_long
    ##################################################

    ### energy density background and Hubble parameter
    H = sqrt(ρ_tot / (3 * M_Pl^2))

    d_log_t_d_η = inv(H * t)
    d_log_ρ_SM_d_η = -4 - this_BKG_coeff * (ρ₄_eq - ρ₄) / ρ_SM + (Γ₄ - Γₓ) * ρ₄ / (H * ρ_SM)
    d_log_ρ₄_d_η = -3 * (1 + w₄) + this_BKG_coeff * (ρ₄_eq - ρ₄) / ρ₄ - Γ₄ / H + exp(-m₄ / Tₓ) * Γₓ * ρₓ / (H * ρ₄)
    d_log_ρₓ_d_η = -4 + Γₓ * ρ₄ / (H * ρₓ) - exp(-m₄ / Tₓ) * Γₓ / H
    ##################################################

    ### number density of mesons evolution ###########
    this_Br_pion_minus = Br_pion_minus(m₄)
    this_Br_pion_plus = Br_pion_plus(m₄)
    this_Br_kaon_minus = Br_kaon_minus(m₄)
    this_Br_kaon_zero_long = Br_kaon_zero_long(m₄)

    d_n_pion_minus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_pion_minus - Γ_pion_minus * n_pion_minus -
                            σv_pion_minus_pton(T_SM) * (1 - Xₙ) * n_B * n_pion_minus
    d_n_pion_plus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_pion_plus - Γ_pion_plus * n_pion_plus -
                            σv_pion_plus_ntop(T_SM) * Xₙ * n_B * n_pion_plus
    d_n_kaon_minus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_kaon_minus - Γ_kaon_minus * n_kaon_minus -
                            σv_kaon_minus_pton(T_SM) * (1 - Xₙ) * n_B * n_kaon_minus -
                            σv_kaon_minus_ntop(T_SM) * Xₙ * n_B * n_kaon_minus
    d_n_kaon_zero_long_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_kaon_zero_long - Γ_kaon_zero_long * n_kaon_zero_long -
                            σv_kaon_zero_long_pton(T_SM) * (1 - Xₙ) * n_B * n_kaon_zero_long -
                            σv_kaon_zero_long_ntop(T_SM) * Xₙ * n_B * n_kaon_zero_long
    ##################################################

    du[1] = d_log_t_d_η
    du[2] = d_Xₙ_d_t / (H * Xₙ)

    du[3] = d_log_ρ_SM_d_η
    du[4] = d_log_ρ₄_d_η
    du[5] = d_log_ρₓ_d_η

    du[6] = iszero(n_pion_minus) ? 0 : d_n_pion_minus_d_t / (H * n_pion_minus)
    du[7] = iszero(n_pion_plus) ? 0 : d_n_pion_plus_d_t / (H * n_pion_plus)
    du[8] = iszero(n_kaon_minus) ? 0 : d_n_kaon_minus_d_t / (H * n_kaon_minus)
    du[9] = iszero(n_kaon_zero_long) ? 0 : d_n_kaon_zero_long_d_t / (H * n_kaon_zero_long)

    typeof(u) == Vector{Float64} && typeof(du) == Vector{Float64} && open("tmp.log", "a+") do io
        println(io, "a: ", a)
        println(io, "T: ", T_SM)
        println(io, "ρ_SM: ", ρ_SM)
        println(io, "ρ₄: ", ρ₄)
        println(io, "ρₓ: ", ρₓ)
        println(io)
    end

    return du
end

In [ ]:
function magic(m₄, U_a4_norm, Brₓ; num_points=1000)
    a_ini = 1

    this_BKG_coeff_peak = BKG_coeff_peak(m₄, U_a4_norm)
    T_ini = 10 * T_crit(m₄, c_ν_e)
    @assert T_ini ≥ m₄
    # T_ini = this_BKG_coeff_peak ≥ 1 ? max(m₄, T_fo(m₄, U_a4_norm, c_ν_e)) : T_crit(m₄, c_ν_e)
    a_end = T_ini / eV(1)
    # a_end = 1e2

    Γ_ν = G_F^2 * m₄^5 * U_a4_norm^2 / (192 * π^3)
    Γ_SM = Γ_ν / Br_ν(m₄)
    Γₓ = Γ_SM * Brₓ / (1 - Brₓ)
    Γ₄ = Γ_SM + Γₓ
    
    t_ini = 2.25 * NU.s * (inv ∘ sqrt)(N_ρ(T_ini) + g₄ * 7 / 8) * (MeV(1) / T_ini)^2 # Eq. (3.56) of Baumann's Cosmology
    Xₙ_ini = inv(1 + exp(Q / T_ini))
    ρ_SM_ini = (π^2 / 30) * N_ρ(T_ini) * T_ini^4
    ρ₄_ini = 1e-30 * ρ_SM_ini
    # ρ₄_ini = this_BKG_coeff_peak ≥ 1 ? numeric_energy_density_FD(T_ini / m₄) * g₄ * m₄^4 : 1e-30 * ρ_SM_ini
    # n₄_ini = numeric_number_density_FD(T_ini / m₄) * g₄ * m₄^3
    ρₓ_ini = 1e-30 * ρ_SM_ini
    ρ_tot_ini = ρ_SM_ini + ρ₄_ini + ρₓ_ini

    n_γ_ini = 2 * ζ₃ * T_ini^3 / π^2
    n_meson_ini = 1e-20 * n_γ_ini

    evolution_problem = ODEProblem(evolution_system!,
        map(log ∘ (v -> EUval(EU, v)),
            [
                t_ini, Xₙ_ini,
                ρ_SM_ini, ρ₄_ini, ρₓ_ini,
                n_meson_ini, n_meson_ini, n_meson_ini, n_meson_ini
            ]
        ),
        (log(a_ini), log(a_end)),
        (m₄, U_a4_norm, Γ₄, Γₓ)
    )
    # evolution_solution = solve(evolution_problem, AutoTsit5(Rodas4P()))
    evolution_solution = solve(evolution_problem, Rodas5P())
    # evolution_solution = solve(evolution_problem, Rodas4P())
    evolution_solution.retcode == SciMLBase.ReturnCode.Success ||
        @warn "$(m₄), $(U_a4_norm), $(Brₓ): $(evolution_solution.retcode)"

    a_list = geomspace(a_ini, a_end, num_points)
    results_list = (evolution_solution ∘ log).(a_list)

    t_list = map((q -> EU(q, -1)) ∘ exp ∘ (v -> v[1]), results_list)
    Xₙ_list = map(exp ∘ (v -> v[2]), results_list)
    ρ_SM_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[3]), results_list)
    ρ₄_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[4]), results_list)
    ρₓ_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[5]), results_list)
    # ρ_tot_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[3]), results_list)
    # ρ_SM_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[4]), results_list)
    # ρ₄_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[5]), results_list)
    # ρₓ_list = ρ_tot_list .- ρ_SM_list .- ρ₄_list

    ρ_tot_list = ρ_SM_list .+ ρ₄_list .+ ρₓ_list
    T_SM_list = map(from_ρ_SM_to_T_SM, ρ_SM_list)

    ρ_γ_list = π^2 / 30 * 2 * T_SM_list.^4
    ρ_ν_list = ρ_γ_list * 7 / 8
    T_ν_decouple_first_index = findfirst(≤(MeV(0.8)), T_SM_list)
    isnothing(T_ν_decouple_first_index) || for ii ∈ T_ν_decouple_first_index+1:length(T_SM_list)
        ρ_ν_list[ii] = ρ_ν_list[T_ν_decouple_first_index] * (a_list[T_ν_decouple_first_index] / a_list[ii])^4
    end

    return a_list,
        t_list, Xₙ_list,
        ρ_tot_list,
        ρ_SM_list, ρ₄_list, ρₓ_list,
        T_SM_list, ρ_γ_list, ρ_ν_list,
        Γ₄, Γ_SM, Γₓ
end

# Benchmark

In [ ]:
m₄, U_e4_norm, Brₓ = GeV(arbfloat_1000(1.08)), arbfloat_1000(1e-5), arbfloat_1000(0.99)
# m₄, U_e4_norm, Brₓ = GeV(1), 1e-4, 0.999
# U_e4_norm = 1e-5
# Brₓ = .99

a_list,
    t_list, Xₙ_list,
    ρ_tot_list,
    ρ_SM_list, ρ₄_list, ρₓ_list,
    T_SM_list, ρ_γ_list, ρ_ν_list,
    Γ₄, Γ_SM, Γₓ = magic(m₄, U_e4_norm, Brₓ)

# ρ_tot_list = ρ_SM_list .+ ρ₄_list .+ ρₓ_list

H_list = sqrt.(ρ_tot_list ./ (3 * M_Pl^2))
@info "Benchmarking done."

In [ ]:
fig = Figure(size=(1000, 400))
ax_1 = Axis(fig[1, 1],
    title=L"$X_n$ Evolution",
    xlabel=L"$T$ [MeV]", ylabel=L"$X_n$",
    xscale=log10, # yscale=log10,
    xreversed=true,
    limits=((1e-2, 1e2), (-1e-2, .6)),
    yticks=-1:.1:1,
)
lines!(ax_1, EUval.(MeV, T_SM_list_SBBN), Xₙ_list_SBBN; label="SBBN")
lines!(ax_1, EUval.(MeV, T_SM_list), Xₙ_list; label="With HNL")
lines!(ax_1, EUval.(MeV, [T_nuc, T_nuc]), [-0, 1]; linestyle=:dash)
axislegend(ax_1; position=:lb)

ax_2 = Axis(fig[1, 2],
    title="Energy Density Evolution",
    xlabel=L"$T$ [MeV]", ylabel=L"$\rho / \rho_\nu$",
    xscale=log10, yscale=log10,
    xticks=exp10.(-10:10), xtickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    yticks=exp10.(-10:10), ytickformat=vs->[L"10^{%$(Int(log10(v)))}" for v in vs],
    xreversed=true,
    limits=((1e-2, maximum(EUval.(MeV, T_SM_list))), (1e-4, 1e2))
)
lines!(ax_2, EUval.(MeV, T_SM_list), ρ_tot_list ./ ρ_ν_list; label=L"\rho_\mathrm{tot} / \rho_\nu",)
lines!(ax_2, EUval.(MeV, T_SM_list), ρ_SM_list ./ ρ_ν_list; label=L"\rho_\mathrm{SM} / \rho_\nu",)
lines!(ax_2, EUval.(MeV, T_SM_list), (ρ₄_list .+ ρₓ_list) ./ ρ_ν_list; label=L"\left(\rho_N + \rho_X\right) / \rho_\nu", linestyle=:dash)
lines!(ax_2, EUval.(MeV, T_SM_list), ρ₄_list ./ ρ_ν_list; label=L"\rho_N / \rho_\nu", linestyle=:dash)
lines!(ax_2, EUval.(MeV, T_SM_list), ρₓ_list ./ ρ_ν_list; label=L"\rho_X / \rho_\nu", linestyle=:dash)
lines!(ax_2, EUval.(MeV, [T_nuc, T_nuc]), [1e-5, 1e6]; linestyle=:dash)
axislegend(ax_2; position=:rb)
fig